I - DATA

In [13]:
from typing import List, Dict
import os
import re
import numpy as np
import pandas as pd
from fredapi import Fred


# Param
FRED_API_KEY = os.getenv("FRED_API_KEY", "2d98c6caa7753b549869a87c5636fea0").strip()

def _is_valid_fred_key(key: str) -> bool:
    return bool(re.fullmatch(r"[a-z0-9]{32}", key))

if not _is_valid_fred_key(FRED_API_KEY):
    raise ValueError(
        "FRED_API_KEY invalide: il faut exactement 32 caractères alphanumériques en minuscules. "
        f"Reçu: '{FRED_API_KEY}' (len={len(FRED_API_KEY)}). "
        "Corrige la clé (souvent un espace en trop)."
    )

series_ids = [
    "EFFR","DFEDTARU","DFEDTARL","IORB",
    "DGS1","DGS2","DGS5","DGS7","DGS10","DGS3MO",
    "T10Y2Y","T10Y3M",
    "NFCI","M2SL","CPIAUCSL","UNRATE",
    "INDPRO","USSLIND","MICH","PPIACO","DCOILWTICO","BUSLOANS",
    "MPRIME","CPILFESL","TCU","UMCSENT"
]

fred = Fred(api_key=FRED_API_KEY)

# fonction qui trouve le dernier point non-NaN dans le mois
def last_valid_in_bucket(s: pd.Series):
    s = s.dropna()
    return s.iloc[-1] if not s.empty else np.nan

# détecter la fréquence officielle FRED
meta_rows = []
for sid in series_ids:
    try:
        info = fred.get_series_info(sid)
        freq = str(info.get("frequency", "")).strip()
        meta_rows.append({"series_id": sid, "frequency": freq})
    except Exception as e:
        # Si la métadonnée échoue, on traite par défaut comme haute fréquence
        meta_rows.append({"series_id": sid, "frequency": "UNKNOWN", "error": repr(e)})

meta = pd.DataFrame(meta_rows)
is_monthly = meta["frequency"].str.lower().str.startswith("monthly")
monthly_ids: List[str] = meta.loc[is_monthly, "series_id"].tolist()
hi_freq_ids: List[str] = meta.loc[~is_monthly, "series_id"].tolist()

# --- Étape 2 : télécharger et construire les deux DataFrame ---
# 2a) Mensuel natif
monthly_series: Dict[str, pd.Series] = {}
for sid in monthly_ids:
    try:
        s = fred.get_series(sid)
        s.index = pd.to_datetime(s.index)
        s = s.resample("M").last()
        monthly_series[sid] = s.rename(sid)
    except Exception as e:
        print(f"[WARN] {sid}: {e}")

df_monthly_native = (
    pd.concat(monthly_series.values(), axis=1).sort_index()
    if monthly_series else pd.DataFrame()
)

# 2b) Daily
hi_freq_series: Dict[str, pd.Series] = {}
for sid in hi_freq_ids:
    try:
        s = fred.get_series(sid)
        s.index = pd.to_datetime(s.index)
        s_eom = s.resample("M").apply(last_valid_in_bucket)
        hi_freq_series[sid] = s_eom.rename(sid)
    except Exception as e:
        print(f"[WARN] {sid}: {e}")

df_monthly_eom_from_highfreq = (
    pd.concat(hi_freq_series.values(), axis=1).sort_index()
    if hi_freq_series else pd.DataFrame()
)

print("Monthly")
print(df_monthly_native.tail(3))
print("\nDaily")
print(df_monthly_eom_from_highfreq.tail(3))


=== Mensuel natif ===
               M2SL  CPIAUCSL  UNRATE    INDPRO  USSLIND  MICH   PPIACO  \
2025-08-31  22108.4   323.364     4.3  103.9203      NaN   4.8  262.443   
2025-09-30  22212.5   324.368     NaN       NaN      NaN   4.7      NaN   
2025-10-31      NaN       NaN     NaN       NaN      NaN   NaN      NaN   

             BUSLOANS  MPRIME  CPILFESL     TCU  UMCSENT  
2025-08-31  2684.6980    7.50   329.793  77.376     58.2  
2025-09-30  2695.2219    7.38   330.542     NaN     55.1  
2025-10-31        NaN    7.23       NaN     NaN      NaN  

=== Haute fréquence agrégée fin de mois (dernier non-NaN) ===
            EFFR  DFEDTARU  DFEDTARL  IORB  DGS1  DGS2  DGS5  DGS7  DGS10  \
2025-09-30  4.09      4.25      4.00  4.15  3.68  3.60  3.74  3.93   4.16   
2025-10-31  3.86      4.00      3.75  3.90  3.70  3.60  3.71  3.89   4.11   
2025-11-30  3.87      4.00      3.75  3.90  3.65  3.57  3.69  3.87   4.11   

            DGS3MO  T10Y2Y  T10Y3M     NFCI  DCOILWTICO  
2025-09-30 

In [22]:
from pathlib import Path
from skfin.dataloaders.cache import CacheManager

CACHE_DIR = Path("data")
CACHE_DIR.mkdir(parents=True, exist_ok=True)

cm = CacheManager(cache_dir=CACHE_DIR)

FILENAME_NATIVE = CACHE_DIR / "FRED_monthly.parquet"
FILENAME_EOM = CACHE_DIR / "FRED_EOM.parquet"

# Sauvegarde correcte des deux DataFrames
cm.save_to_cache(df_monthly_native, FILENAME_NATIVE)
cm.save_to_cache(df_monthly_eom_from_highfreq, FILENAME_EOM)
